# Figure 7: Climate uncertainty

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

os.chdir('/home/rooda/Dropbox/Patagonia/')

## Data

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip")
basins = basins[["basin_id",'basin_name', "geometry"]].set_index("basin_id")

data = pd.read_csv("MS2 Results/dataset_climate_future.csv", index_col = "ID")
area = pd.read_csv("MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id").area_RGI6

basins['basin_name'] = basins['basin_name'].replace({'Santa Cruz': 'Santa Cruz                          '})
basins = pd.concat([basins, area, data], axis=1)
basins["area_RGI6_percent"] = basins["area_RGI6"]/basins.area_RGI6.sum()

# major catchments
basins_m = basins.dropna(subset = ['basin_name'])

In [ ]:
# weighted-means
df_wm = {'PRSN': [(basins.PRSN_climate * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                  (basins.PRSN_ssp     * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                  (basins.PRSN_gcm     * basins.area_RGI6).sum() / basins.area_RGI6.sum(),
                  (basins.PRSN_bcm     * basins.area_RGI6).sum() / basins.area_RGI6.sum()], 
         'T2M':  [(basins.T2M_climate  * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                  (basins.T2M_ssp      * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                  (basins.T2M_gcm      * basins.area_RGI6).sum() / basins.area_RGI6.sum(),
                  (basins.T2M_bcm      * basins.area_RGI6).sum() / basins.area_RGI6.sum()], 
         'PPD':  [(basins.PPD_climate  * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                  (basins.PPD_ssp      * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                  (basins.PPD_gcm      * basins.area_RGI6).sum() / basins.area_RGI6.sum(),
                  (basins.PPD_bcm      * basins.area_RGI6).sum() / basins.area_RGI6.sum()]}

df_wm = pd.DataFrame(data=df_wm, index=["Climate", "SSP", "GCM", "BCM"])

## Plot elements

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -71.7, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+ x +'</b>' for x in names]

## Plot

In [ ]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.02, column_widths = [0.36, 0.36, 0.31], shared_xaxes = True,
                    subplot_titles = ["Precipitation (PP) ensemble uncertainty","Temperature (T2M) ensemble uncertainty", "Individual uncertainty"],
                    specs=[[{"type": "scattergeo", "rowspan": 3}, {"type": "scattergeo", "rowspan": 3}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

cl = px.colors.colorbrewer.RdYlBu
cs = px.colors.colorbrewer.GnBu
cd = px.colors.colorbrewer.Dark2

## Basemap
for x in range(1,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x)


# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_ensemble_std'], 
                            colorscale = [cs[2], cs[4], cs[8]], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 500, zmax = 2500, colorbar=dict(len=0.45, x=0.24, y= 0.77, title='σ PP', ticksuffix = " mm", thickness=20)), row=1, col=1)
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins[basins.PP_ensemble_cv > 0.40].index, z = basins[basins.PP_ensemble_cv > 0.40].PP_ensemble_cv, 
                           colorscale = ["rgba(213,213,213,0)", "rgba(213,213,213,0)"], marker_line_color ='black', showscale= False, marker_line_width=0.3), row=1, col=1)
fig.add_annotation(text="(a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)


# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_ensemble_std'], 
                            colorscale=["#ffe9ba", cl[3],  cl[1]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = 0.7, zmax = 2, colorbar=dict(len=0.45, x=0.61, y= 0.77, title='σ T2M', ticksuffix = " ºC", thickness=20, dtick= 0.5)), row=1, col=2)
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins[basins.T2M_ensemble_cv > 0.40].index, z = basins[basins.T2M_ensemble_cv > 0.4].PP_ensemble_cv, 
                           colorscale = ["rgba(213,213,213,0)", "rgba(213,213,213,0)"], marker_line_color ='black', showscale= False, marker_line_width=0.3), row=1, col=2)
fig.add_annotation(text="(b)", font=dict(size=16), x=0.368, y=0.995,  xref = "paper", yref = "paper", showarrow=False)


# layout a) and b) ---------------------------------------------------------------------------------------------------------------------
for x in range(1,3):
    ## Add basin and hydrological zone names plus the hydro zone divides
    fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.8, color = 'black'), opacity = 0.7, showlegend = False),row=1, col=x)  
    fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.7)"), showlegend = False),row=1, col=x)
    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['basin_name'], mode = 'text', showlegend = False,
                       textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=x)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)


# Solid precipitation spread (c) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins.sort_values("PRSN_climate").PRSN_climate, x = basins.sort_values("PRSN_climate").area_RGI6_percent.cumsum(), 
                         marker_color = cd[1],  showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PRSN_ssp").PRSN_ssp, x = basins.sort_values("PRSN_ssp").area_RGI6_percent.cumsum(), 
                         marker_color = cd[6],  showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PRSN_gcm").PRSN_gcm, x = basins.sort_values("PRSN_gcm").area_RGI6_percent.cumsum(), 
                         marker_color = cd[5],   showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PRSN_bcm").PRSN_bcm, x = basins.sort_values("PRSN_bcm").area_RGI6_percent.cumsum(), 
                         marker_color = cd[2],  showlegend = False), row=1, col=3)

fig.update_yaxes(title_text="σ Solid precpitation (mm)", side = "right", title_standoff = 2, row = 1, col = 3)
fig.update_yaxes(range = [0,1000], tickangle = 0, row = 1, col = 3)
fig.add_annotation(text="(c)", font=dict(size=16), x=0.03, y=950, showarrow=False, row=1, col=3)
fig.add_annotation(text="Climate ({:.0f}mm)".format(df_wm.PRSN["Climate"]), font=dict(size=13, color = cd[1]), x=0.55, y=550, textangle=-23, showarrow=False, row=1, col=3)
fig.add_annotation(text="SSP ({:.0f}mm)".format(df_wm.PRSN["SSP"]), font=dict(size=13, color = cd[6]), x=0.80, y=280, textangle=-20, showarrow=False, row=1, col=3)
fig.add_annotation(text="GCM ({:.0f}mm)".format(df_wm.PRSN["GCM"]), font=dict(size=13, color = cd[5]), x=0.20, y=70, ay=-60, textangle=0, showarrow=True, arrowhead=0, arrowcolor="#636363", row=1, col=3)
fig.add_annotation(text="BCM ({:.0f}mm)".format(df_wm.PRSN["BCM"]), font=dict(size=13, color = cd[2]), x=0.50, y=50, textangle=-5, showarrow=True, arrowhead=0, arrowcolor="#636363", row=1, col=3)

# Temperature spread (d) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins.sort_values("T2M_climate").T2M_climate, x = basins.sort_values("T2M_climate").area_RGI6_percent.cumsum(), 
                         marker_color = cd[1],  showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("T2M_ssp").T2M_ssp, x = basins.sort_values("T2M_ssp").area_RGI6_percent.cumsum(), 
                         marker_color = cd[6],  showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("T2M_gcm").T2M_gcm, x = basins.sort_values("T2M_gcm").area_RGI6_percent.cumsum(), 
                         marker_color = cd[5],   showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("T2M_bcm").T2M_bcm, x = basins.sort_values("T2M_bcm").area_RGI6_percent.cumsum(), 
                         marker_color = cd[2],  showlegend = False), row=2, col=3)

fig.update_yaxes(title_text="σ Temperature (ºC)", side = "right", title_standoff = 5, row = 2, col = 3)
fig.update_yaxes(range = [0, 2], tickangle = 0, row = 2, col = 3)
fig.add_annotation(text="(d)", font=dict(size=16), x=0.03, y=1.9, showarrow=False, row=2, col=3)
fig.add_annotation(text="Climate ({:.1f}°C)".format(df_wm.T2M["Climate"]), font=dict(size=13, color = cd[1]), x=0.5, y=1.2, textangle=-15, showarrow=False, row=2, col=3)
fig.add_annotation(text="SSP ({:.1f}°C)".format(df_wm.T2M["SSP"]), font=dict(size=13, color = cd[6]), x=0.5, y=0.8, textangle=-5, showarrow=False, row=2, col=3)
fig.add_annotation(text="GCM ({:.1f}°C)".format(df_wm.T2M["GCM"]), font=dict(size=13, color = cd[5]), x=0.5, y=0.5, textangle=-5, showarrow=False, row=2, col=3)
fig.add_annotation(text="BCM ({:.1f}°C)".format(df_wm.T2M["BCM"]), font=dict(size=13, color = cd[2]), x=0.5, y=0.2, textangle=-5, showarrow=False, row=2, col=3)

# Positive degree-day sum spread (e) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins.sort_values("PPD_climate").PPD_climate, x = basins.sort_values("PPD_climate").area_RGI6_percent.cumsum(), 
                         marker_color = cd[1],  showlegend = False), row=3, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PPD_ssp").PPD_ssp, x = basins.sort_values("PPD_ssp").area_RGI6_percent.cumsum(), 
                         marker_color = cd[6],  showlegend = False), row=3, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PPD_gcm").PPD_gcm, x = basins.sort_values("PPD_gcm").area_RGI6_percent.cumsum(), 
                         marker_color = cd[5],   showlegend = False), row=3, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PPD_bcm").PPD_bcm, x = basins.sort_values("PPD_bcm").area_RGI6_percent.cumsum(), 
                         marker_color = cd[2],  showlegend = False), row=3, col=3)

fig.update_xaxes(range = [-0.02,1.02], title_text="Accumulated glacier area (%)", title_standoff = 0, dtick = 0.25, tickangle = 0, tickformat = ',.0%', row = 3, col = 3)
fig.update_yaxes(title_text="σ Positive degree-day sum (ºC)", side = "right", title_standoff = 5, row = 3, col = 3)
fig.update_yaxes(range = [0,500], tickangle = 0, row = 3, col = 3)
fig.add_annotation(text="(e)", font=dict(size=16), x=0.03, y=470, showarrow=False, row=3, col=3)
fig.add_annotation(text="Climate ({:.0f}°C)".format(df_wm.PPD["Climate"]), font=dict(size=13, color = cd[1]), x=0.5, y=300, textangle=-10, showarrow=False, row=3, col=3)
fig.add_annotation(text="SSP ({:.0f}°C)".format(df_wm.PPD["SSP"]), font=dict(size=13, color = cd[6]), x=0.5, y=220, textangle=-10, showarrow=False, row=3, col=3)
fig.add_annotation(text="GCM ({:.0f}°C)".format(df_wm.PPD["GCM"]), font=dict(size=13, color = cd[5]), x=0.5, y=160, textangle=-5, showarrow=False, row=3, col=3)
fig.add_annotation(text="BCM ({:.0f}°C)".format(df_wm.PPD["BCM"]), font=dict(size=13, color = cd[2]), x=0.5, y=70, textangle=-2, showarrow=False, row=3, col=3)

# layout c, d and e
fig.update_traces(line = dict(width = 2), opacity = 0.8, row = 1, col = 3)
fig.update_traces(line = dict(width = 2), opacity = 0.8, row = 2, col = 3)
fig.update_traces(line = dict(width = 2), opacity = 0.8, row = 3, col = 3)
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general
fig.update_layout(autosize = False, template = "seaborn", width = 1000, height = 670, margin = dict(l=10, r=5, b=5, t=30, pad=0, autoexpand=True))
fig.show()

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_7_climate_uncertainty.png", scale=4)

In [ ]:
"The standard deviation of the mean annual precipitation in the long term (2070–2099) was greater than 1,000 mm in {:.1f}% of the glacier area".format(
    basins[basins["PP_ensemble_std"] > 1000].area_RGI6.sum() * 100 / basins.area_RGI6.sum())

In [ ]:
"Similarly, the standard deviation of the temperature was greater than 1.0 ºC in {:.1f}% of the glacier area".format(
    basins[basins["T2M_ensemble_std"] > 1].area_RGI6.sum() * 100 / basins.area_RGI6.sum())